In [1]:
import math
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute


In [2]:
# The starting pattern is represented by this list of numbers.
# Please use it as an input for your solution.
lights = [0, 1, 1, 1, 0, 0, 1, 1, 1]
# lights = [1, 0, 1, 1]
def isSquare(n):
    i = 1
    while i * i < n:
        i += 1
    return i * i == n
def week2a_ans_func(lights):
    ##### build your quantum circuit here
    
    # weird that only lights are sent, what if the lights are not a square-matrix?
    n = len(lights)
    nn = 2 * n
    nnn = 3 * n
    if not isSquare(n):
        return None
    m = int(math.floor(math.sqrt(n)))
#     print(m)
#     print(type(m))
    q = QuantumRegister(n * 3 + 1, 'q')
    c = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(q, c)
    mc = [i + n for i in range(n) if lights[i]]
    ors = [i + n for i in range(n) if lights[i] == 0]
    anc = list(range(nn, nnn - 2))
    # make last one the flip |->
    qc.x(nnn)
    qc.h(nnn)
    # continue
    # superposition
    for i in range(n):
        qc.h(i)
    qc.barrier()
    # Oracle
    def orac():
        def check():
            for i in range(n):
                qc.cx(i, i + n)
                j = i % m
                if (j < (j + 1) % m): 
                    qc.cx(i, i + 1 + n)
                if ((j - 1 + m) % m < j):
                    qc.cx(i, i - 1 + n)
                if (i + m < n):
                    qc.cx(i, i + m + n)
                if (i - m >= 0):
                    qc.cx(i, i - m + n)
        def icheck():
            for i in reversed(range(n)):
                j = i % m
                if (i - m >= 0):
                    qc.cx(i, i - m + n)
                if (i + m < n):
                    qc.cx(i, i + m + n)
                if ((j - 1 + m) % m < j):
                    qc.cx(i, i - 1 + n)
                if (j < (j + 1) % m): 
                    qc.cx(i, i + 1 + n)
                qc.cx(i, i + n)
                #qc.barrier()
        check()
    #     print('Ors {}'.format(ors))
    #     print(mc)
    #     print(anc)
    #     print(nnn - 2)
        qc.mct(mc, nnn - 2, anc, 'basic')
        qc.barrier()
        for x in ors:
            qc.x(x)
        qc.mct(ors, nnn - 1, anc, 'basic')
        qc.barrier()
    #     # make it |-> maybe do it earlier :)
    #     qc.x(nnn)
    #     qc.h(nnn)
        # this should flip the sign
        qc.ccx(nnn-2, nnn-1, nnn)
        qc.barrier()
        # now i need to reverse everything
        qc.mct(ors, nnn - 1, anc, 'basic')
        for x in ors:
            qc.x(x)
        qc.barrier()
        qc.mct(mc, nnn - 2, anc, 'basic')
        icheck()
        qc.barrier()
    def diffuse():
        for i in range(n):
            qc.h(i)
        for i in range(n):
            qc.x(i)
        qc.barrier()
        qc.h(n - 1)
        qc.mct(list(range(n - 1)), n - 1, anc, 'basic')
        qc.h(n - 1)
        qc.barrier()
        for i in range(n):
            qc.x(i)
        for i in range(n):
            qc.h(i)
        qc.barrier()
    ########## now try measuring to check how it is going
    mm = math.floor(math.pi / 4 * 2 ** (n/2))
    #print(mm) #use mm if you want maximum amplitude, but less if you don't care :)
    for i in range(min(6,mm)):
        orac()
        diffuse()
    for i in range(n):
        qc.measure(i, i)
    
    #####  In addition, please make it a function that can solve the problem even with different inputs (lights). We do validation with different inputs.
    qc = qc.reverse_bits()
    return qc
qc = week2a_ans_func(lights)
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=101)
result = job.result()
count =result.get_counts()
print(count)
# qc.draw('mpl')

{'000010000': 1, '100001110': 1, '100100101': 1, '100100110': 1, '100101011': 1, '100111011': 1, '100111101': 1, '100111110': 1, '101000000': 1, '101001011': 1, '101011011': 1, '101101000': 1, '101101011': 1, '000010111': 1, '101110110': 1, '110000011': 1, '110010000': 1, '110010101': 39, '110010111': 1, '110011010': 1, '110011100': 1, '110011110': 1, '000011010': 1, '110100110': 1, '110110001': 1, '110110110': 1, '110111101': 1, '111010000': 1, '111010100': 1, '111111101': 1, '000100110': 1, '000101000': 1, '000101110': 1, '000110110': 1, '000111111': 1, '001000010': 1, '001000111': 1, '001001000': 1, '001001110': 1, '001001111': 1, '001110010': 1, '001110110': 1, '001111010': 1, '010000011': 1, '010000101': 1, '010000110': 1, '010001000': 1, '010011100': 1, '010011101': 1, '010100001': 1, '010100010': 1, '010100011': 1, '010101001': 1, '010110111': 1, '010111010': 1, '011000001': 1, '011000011': 1, '011000110': 1, '011010010': 1, '011101000': 1, '011101011': 1, '011101110': 1, '01111